In [1]:
import numpy as np
import numba as nb
from numba import cuda, float32

In [16]:
# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit(device=False)
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        #for j in range(TPB):
        #    tmp += sA[tx, j] * sB[j, ty]
        tmp = 7.0 # tx*1000+ty
        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [12]:
        
A = np.zeros((5,5),dtype=np.float32)  
i = np.arange(A.shape[0])
A[i,i]=2
B=np.zeros_like(A)
C=np.zeros_like(B)
B[i,(i+1)%A.shape[1]] = 3


threads_per_block=(32,32)
tpb = threads_per_block[0]*threads_per_block[1]
xblocks = np.int( (A.shape[0]+threads_per_block[0]-1)/threads_per_block[0])
yblocks = np.int( (B.shape[1]+threads_per_block[1]-1)/threads_per_block[1]) 

blocks_per_grid=(xblocks,yblocks)


A=nb.SmartArray(A)
B=nb.SmartArray(B)
C=nb.SmartArray(C)

In [13]:
blocks_per_grid

(1, 1)

In [17]:
fast_matmul[blocks_per_grid,threads_per_block](A,B,C)
np.asarray(C)
#print(np.matmul[blocks_per_grid,threads_per_block](B,B.T))

CudaAPIError: [700] Call to cuMemcpyDtoH results in UNKNOWN_CUDA_ERROR